# Imports

removed checking if in CoLab

In [ ]:
#imports
from platform import python_version

#basic python stuff
import os
import json
from pathlib import Path

#basics from the SciPy Stack
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#colab stuff
from google.colab import drive

#data managing
from sklearn.model_selection import train_test_split
from skimage import io #read in images
from skimage.transform import resize

#model
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras import layers, callbacks
from keras.optimizers import Adam

#progress bar
from tqdm.notebook import tqdm

In [ ]:
# settings

# implements progress_apply into pandas
tqdm.pandas(desc='Pandas_Progress')

In [ ]:
print("Tensorflow version", tf.__version__)
print("Python version =",python_version())

In [ ]:
# get access to google drive
drive.mount('/content/drive')

In [ ]:
# import local python files
import importlib.util

# https://github.com/maxvfischer/keras-image-segmentation-loss-functions
spec = importlib.util.spec_from_file_location("binary_losses", "/content/drive/MyDrive/ML_Project_Satellite_Images/binary_losses.py")
binary_losses = importlib.util.module_from_spec(spec)
spec.loader.exec_module(binary_losses)

# Hyperparameters

In [ ]:
# training parameters
sample_size = 2000
batch_size = 128
epochs = 10
test_size = 0.3
val_size = 0.3

random_state = 42

# image preprocessing
img_size = 128
anti_aliasing = True
resize_method = tf.image.ResizeMethod.BILINEAR
mask_threshold = 0.5

# model parameters
params = {
    'learning_rate' : 0.01,
    'loss' : 'binary_crossentropy',
    'metrics' : ['accuracy']
  }

# for model saving
model_name = 'alternative_neural_network'

# Retrieve the Dataset

In [ ]:
# unzip the dataset
!unzip -n -q /content/drive/MyDrive/ML_Project_Satellite_Images/data/current_dataset.zip -d /content/

In [ ]:
# read in samples.csv with information about the images (only a sample)
samples_df = pd.read_csv('/content/dataset/samples.csv').sample(sample_size, random_state=random_state)
samples_df.set_index('id', inplace=True)
samples_df

In [ ]:
# paths to the sat/mask folder
path_sat_folder = '/content/dataset/images/satellite/'
path_mask_folder = '/content/dataset/images/mask/'

In [ ]:
# append absoulute paths of the images to the dataframe
samples_df['abs_satellite_path'] = samples_df['satellite_file'].apply(lambda x: path_sat_folder+x)
samples_df['abs_mask_path'] = samples_df['mask_file'].apply(lambda x: path_mask_folder+x)

# Prepare the tf.data.Dataset

- https://towardsdatascience.com/what-is-the-best-input-pipeline-to-train-image-classification-models-with-tf-keras-eb3fe26d3cc5  
- https://www.tensorflow.org/guide/data_performance


In [ ]:
def parse_satellite_image(file_path):
  img = tf.io.read_file(file_path)
  img = tf.io.decode_jpeg(img, channels=3)
  img = tf.image.resize(img, [img_size, img_size], method=resize_method, antialias=anti_aliasing)
  img = img / 255.
  return img

In [ ]:
def parse_mask_image(file_path):
  img = tf.io.read_file(file_path)
  img = tf.io.decode_png(img, channels=4)
  img = tf.image.resize(img, [img_size, img_size], method=resize_method, antialias=anti_aliasing)
  img = img[:,:,2]
  img = img / 255.
  mask = tf.greater(img,mask_threshold)
  img = tf.where(mask, 1, 0)
  return img

In [ ]:
def make_dataset(samples_df, batch_size):
  # paths as Dataset
  satellite_paths_ds = tf.data.Dataset.from_tensor_slices(samples_df['abs_satellite_path'])
  mask_paths_ds = tf.data.Dataset.from_tensor_slices(samples_df['abs_mask_path'])
  # images as Dataset
  satellite_ds = satellite_paths_ds.map(parse_satellite_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  mask_ds = mask_paths_ds.map(parse_mask_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  # combine inputs and targets https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit
  ds = tf.data.Dataset.zip((satellite_ds, mask_ds))
  # configure the Dataset for better performance
  ds = ds.batch(batch_size)
  ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  ds = ds.cache()

  return ds

# Split Training, Validation and Test Data

In [ ]:
# split in (train+val) and test
samples_df_train_val, samples_df_test = train_test_split(samples_df, test_size=test_size, random_state=random_state)
# split (train+val)
samples_df_train, samples_df_val = train_test_split(samples_df_train_val, test_size=val_size, random_state=random_state)

In [ ]:
print('Training Shape: ',samples_df_train.shape)
print('Validation Shape: ',samples_df_val.shape)
print('Test Shape: ',samples_df_test.shape)

In [ ]:
train_ds = make_dataset(samples_df_train, batch_size)
val_ds = make_dataset(samples_df_val, batch_size)
test_ds = make_dataset(samples_df_test, batch_size)

# Test Feature Extraction

In [ ]:
def show_images(X,Ys,names,fig_height=4):
  'Plots X and multiple Y'
  if not isinstance(Ys,list):
    Ys = [Ys]
  if not isinstance(names,list):
    names = [names]
  cols = 1 + len(Ys)
  for i in range(X.shape[0]):
    fig,axs = plt.subplots(1,cols,figsize=(fig_height*cols,fig_height))
    axs[0].axis('off')
    axs[0].imshow(X[i])
    axs[0].set_title('Satellite')
    for j,(Y,name) in enumerate(zip(Ys,names)):
      axs[j+1].axis('off')
      axs[j+1].imshow(Y[i])
      axs[j+1].set_title(name)
    plt.show()

In [ ]:
class FeatureExtractionLayer(layers.Layer):
  def call(self, inputs):
    # gray image
    inputs_gray = tf.image.rgb_to_grayscale(inputs)
    # gradient
    dy, dx = tf.image.image_gradients(inputs)
    grad = tf.stack([dx,dy],axis=-1)
    grad = tf.norm(grad, axis=-1)
    grad = grad * 2
    # gray gradient
    grad_gray = tf.image.rgb_to_grayscale(grad)
    # combine all
    all_features = tf.concat([inputs,inputs_gray,grad,grad_gray],axis=-1)
    return all_features

In [ ]:
X = list(train_ds.take(1))[0][0]
Y = list(train_ds.take(1))[0][1]
X.shape, Y.shape

In [ ]:
X_features = FeatureExtractionLayer()(X[:5])
X_features.shape

In [ ]:
tf.math.reduce_max(X_features,axis=(0,1,2))

In [ ]:
X_satellite = X_features[...,0:3]
X_satellite_gray = X_features[...,3]
X_grad = X_features[...,4:7]
X_grad_gray = X_features[...,7]
X_satellite.shape, X_satellite_gray.shape, X_grad.shape, X_grad_gray.shape

In [ ]:
X = X_satellite
Ys = [X_satellite_gray,X_grad,X_grad_gray]
names = ['Satellite Gray','Gradient','Gradient Gray']
show_images(X,Ys,names)

## Build alternative model


In [ ]:
def build_model(params):
  inputs = layers.Input((img_size, img_size, 3))

  # feature extraction
  last = FeatureExtractionLayer()(inputs)

  # Neural Network
  last = layers.Reshape((-1, last.shape[-1]))(last)
  last = layers.Dense(256, activation='relu')(last)
  last = layers.Dense(256, activation='relu')(last)
  last = layers.Dense(1, activation='sigmoid')(last)
  
  outputs = layers.Reshape((img_size,img_size))(last)
  
  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  optimizer = Adam(learning_rate=params['learning_rate'])
  optimizer = Adam(learning_rate=0.1)
  model.compile(optimizer=optimizer, loss=params['loss'], metrics=params['metrics'])
  
  return model

In [ ]:
model = build_model(params)
model.summary()

In [ ]:
results = model.fit(train_ds, validation_data=val_ds, batch_size=batch_size, epochs=epochs)

In [ ]:
def plot_history(network_history,figsize=(8,5)):
  history_keys = [key for key in network_history.history.keys() if not key.startswith('val_')]
  for key in history_keys:
    plt.figure(figsize=figsize)
    plt.plot(network_history.history[key])
    plt.plot(network_history.history[f'val_{key}'])
    plt.title(f'model {key}')
    plt.ylabel(key)
    plt.xlabel('epoch')
    plt.legend(['training', 'validation'], loc='upper right')
    plt.tight_layout()
    plt.show()

In [ ]:
plot_history(results)

# Functions for Evaluation

In [ ]:
# function for image reading
def read_satellite_img(filepath):
  img = io.imread(filepath)
  img = resize(img, output_shape=(img_size,img_size), anti_aliasing=anti_aliasing, preserve_range=True)
  img = img / 255.
  return img

def read_mask_img(filepath):
  img = io.imread(filepath)
  if len(img.shape) > 2:
    img = img[:,:,2]
  img = resize(img, output_shape=(img_size,img_size), anti_aliasing=anti_aliasing, preserve_range=True)
  img = img / 255.
  mask = img > mask_threshold
  img[mask] = 1
  img[~mask] = 0
  return img

In [ ]:
# function to load a batch of images
def load_img_batch(samples_df,ids):
  satellite_imgs = samples_df.loc[ids,'abs_satellite_path'].progress_apply(read_satellite_img)
  mask_imgs = samples_df.loc[ids,'abs_mask_path'].progress_apply(read_mask_img)

  satellite_imgs = np.stack(satellite_imgs.to_numpy())
  mask_imgs = np.stack(mask_imgs.to_numpy())

  return satellite_imgs, mask_imgs

In [ ]:
# function to show some samples (with or without the predictions)
def show_sample(X, Y, samples_df, ids, Y_pred=None, threshold=None, sample_size=10, fig_height=4):
  rnd_sample_indices = np.random.random_integers(low=0,high=X.shape[0]-1,size=sample_size)
  cols = 2 if Y_pred is None else 3
  if Y_pred is None:
    cols = 2
  elif threshold is not None:
    cols = 4
  else:
    cols = 3

  for i in rnd_sample_indices:
    fig, axs = plt.subplots(1,cols, figsize=(fig_height*cols,fig_height))
    axs[0].set_title(f'Country: {samples_df.loc[ids[i],"country"]}')
    axs[0].imshow(X[i])
    axs[1].set_title('Given Mask')
    axs[1].imshow(Y[i])
    if Y_pred is not None:
      axs[2].set_title('Prediction')
      axs[2].imshow(Y_pred[i])
      if threshold is not None:
        Y_pred_mask = Y_pred[i] >= threshold
        Y_pred[i,Y_pred_mask] = 1
        Y_pred[i,~Y_pred_mask] = 0
        axs[3].set_title(f'Prediction with threshold = {threshold}')
        axs[3].imshow(Y_pred[i])

    for ax in axs:
      ax.set_xticks([])
      ax.set_yticks([])
    fig.tight_layout()

# Evaluate Training

In [ ]:
X_train, Y_train = load_img_batch(samples_df,samples_df_train.index)

In [ ]:
X_train.shape, Y_train.shape

In [ ]:
Y_pred = model.predict(train_ds)

In [ ]:
type(Y_pred),Y_pred.shape

In [ ]:
np.unique(Y_pred)

In [ ]:
plt.hist(Y_pred[Y_train == 1].flatten(),histtype='step',bins=30,label='Water')
plt.hist(Y_pred[Y_train == 0].flatten(),histtype='step',bins=30,label='No Water')
plt.legend();

In [ ]:
show_sample(X_train,Y_train,samples_df,samples_df_train.index,Y_pred,threshold=0.2)

# Save Model

In [ ]:
#model.save(f'/content/drive/MyDrive/ML_Project_Satellite_Images/models/{model_name}.h5')

In [ ]:
sample_ids = {
    'train_ids':samples_df_train.index.to_list(),
    'val_ids':samples_df_val.index.to_list(),
    'test_ids':samples_df_test.index.to_list()
  }

In [ ]:
with open(f'/content/drive/MyDrive/ML_Project_Satellite_Images/models/{model_name}.json', 'w') as f:
    #json.dump(sample_ids, f)

drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')